In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
def direction(val):
    val1 = Conversion().convert_int256_bytes_to_int(val, signed=False) 
    val2 = Conversion().convert_int256_bytes_to_int(val, signed=True)
    return val1 != val2

In [3]:
chain = Net.POLYGON
platform = Platform.UNIV3
contract = JSONContract.UniswapV3Pool

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [4]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
swap_filt = Filter.create_filter(address=None, event_types=[Pair.events.Burn])  # Listen events from any smart contract

In [5]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [6]:
start = start - 50

In [7]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=swap_filt)

In [8]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    event_signature, owner, tick_lower, tick_upper = evt["topics"]
    args = Conversion().decode_data(evt["data"])
    
    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['owner'] = Conversion().convert_uint256_hex_string_to_address(owner)
    event['details']['tick_lower'] = Conversion().convert_uint256_string_to_int(tick_lower, signed=True)
    event['details']['tick_upper'] = Conversion().convert_uint256_string_to_int(tick_upper, signed=True)    
    if(len(args) == 4):
        event['details']['liquidity_amount'] = Conversion().convert_int256_bytes_to_int(args[1])
        event['details']['amount0'] = Conversion().convert_int256_bytes_to_int(args[2])
        event['details']['amount1'] = Conversion().convert_int256_bytes_to_int(args[3])
    events[k] = event
    
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")

Swap at block:60,958,085 tx:0x26dd36211caa834827265017f5f40e6f32d32128c9b13434cb3c1629abacaab9
Swap at block:60,958,089 tx:0x2a0ed764ba3456eed06afb8b7e460d133db5ed5326de91939092c01f45976145
Swap at block:60,958,105 tx:0x47303492f52eb2fd0f3cb22c0a3c6b831e8a132973f6ca86010125ea45474d49
Swap at block:60,958,114 tx:0xc521cc4cb04e0ddc9c9d3bcf097cc263c5a4b1061c7b25896757d852b5a5d729
Swap at block:60,958,125 tx:0x1f6abd4695b7e4f1a82b4ac342090f79e29f464edec8c132a1ef18013fa32331
Swap at block:60,958,125 tx:0x1f6abd4695b7e4f1a82b4ac342090f79e29f464edec8c132a1ef18013fa32331
Swap at block:60,958,125 tx:0x1f6abd4695b7e4f1a82b4ac342090f79e29f464edec8c132a1ef18013fa32331
.


In [9]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv3pool,burn,uniswap_v3,0x2d9d73c95ecd9aef79cc25fd0ccbb6c29e576b84,0x26dd36211caa834827265017f5f40e6f32d32128c9b1...,60958085,1724452159,{'web3_type': <class 'web3._utils.datatypes.Bu...
1,polygon,uniswapv3pool,burn,uniswap_v3,0xc4c06c9a239f94fc0a1d3e04d23c159ebe8316f1,0x2a0ed764ba3456eed06afb8b7e460d133db5ed5326de...,60958089,1724452167,{'web3_type': <class 'web3._utils.datatypes.Bu...
2,polygon,uniswapv3pool,burn,uniswap_v3,0x95d2483d2a0fff034004f91c53d649623d993896,0x47303492f52eb2fd0f3cb22c0a3c6b831e8a132973f6...,60958105,1724452201,{'web3_type': <class 'web3._utils.datatypes.Bu...
3,polygon,uniswapv3pool,burn,uniswap_v3,0xb6e57ed85c4c9dbfef2a68711e9d6f36c56e0fcb,0xc521cc4cb04e0ddc9c9d3bcf097cc263c5a4b1061c7b...,60958114,1724452221,{'web3_type': <class 'web3._utils.datatypes.Bu...
4,polygon,uniswapv3pool,burn,uniswap_v3,0xa6aedf7c4ed6e821e67a6bfd56fd1702ad9a9719,0x1f6abd4695b7e4f1a82b4ac342090f79e29f464edec8...,60958125,1724452243,{'web3_type': <class 'web3._utils.datatypes.Bu...
5,polygon,uniswapv3pool,burn,uniswap_v3,0xa6aedf7c4ed6e821e67a6bfd56fd1702ad9a9719,0x1f6abd4695b7e4f1a82b4ac342090f79e29f464edec8...,60958125,1724452243,{'web3_type': <class 'web3._utils.datatypes.Bu...
6,polygon,uniswapv3pool,burn,uniswap_v3,0xa6aedf7c4ed6e821e67a6bfd56fd1702ad9a9719,0x1f6abd4695b7e4f1a82b4ac342090f79e29f464edec8...,60958125,1724452243,{'web3_type': <class 'web3._utils.datatypes.Bu...


In [10]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv3pool',
  'type': 'burn',
  'platform': 'uniswap_v3',
  'address': '0x2d9d73c95ecd9aef79cc25fd0ccbb6c29e576b84',
  'tx_hash': '0x26dd36211caa834827265017f5f40e6f32d32128c9b13434cb3c1629abacaab9',
  'blk_num': 60958085,
  'timestamp': 1724452159,
  'details': {'web3_type': web3._utils.datatypes.Burn,
   'owner': '0xC36442b4a4522E871399CD717aBDD847Ab11FE88',
   'tick_lower': 293100,
   'tick_upper': 293940}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv3pool',
  'type': 'burn',
  'platform': 'uniswap_v3',
  'address': '0xc4c06c9a239f94fc0a1d3e04d23c159ebe8316f1',
  'tx_hash': '0x2a0ed764ba3456eed06afb8b7e460d133db5ed5326de91939092c01f45976145',
  'blk_num': 60958089,
  'timestamp': 1724452167,
  'details': {'web3_type': web3._utils.datatypes.Burn,
   'owner': '0xC36442b4a4522E871399CD717aBDD847Ab11FE88',
   'tick_lower': -26520,
   'tick_upper': -20520}},
 2: {'chain': 'polygon',
  'contract': 'uniswapv3pool',
  'type': 'burn',
  'platf